# 실습

1) WordCounter 구현

2) 타이타닉 데이터 분석!

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 13.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=155d83d8d42b7917cc3e1f796b89065ac160bb5b3526ff20dac85d6d2be1927a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [4]:
from pyspark import SparkContext

sc = SparkContext()

## WordCounter 구현하기

- `reduceByKey` 이용

In [5]:
text = sc.parallelize('hello python')

# map 함수를 적용한 RDD 구하기
text_1 = text.filter(lambda x: x != " ")
text_2 = text_1.map(lambda x:(x, 1))

#reduceByKey 함수를 적용한 Word Counter 출력
word_count = text_2.reduceByKey(lambda accum, n: accum + n)  
word_count.collect()

[('h', 2),
 ('l', 2),
 ('p', 1),
 ('y', 1),
 ('e', 1),
 ('o', 2),
 ('t', 1),
 ('n', 1)]

## 타이타닉 데이터분석!

- 생존자와 사망자로 나누어 평균연령을 구해 봅니다.
`map` 함수 이용!

In [26]:
csv_path = '/content/sample_data/train.csv'
csv_data_0 = sc.textFile(csv_path)
csv_data_1 = csv_data_0.filter(lambda line: len(line)>1).map(lambda line: line.split(","))   
columns = csv_data_1.take(1)
csv_data_2 = csv_data_1.filter(lambda line: line[0].isdecimal())
# Name에서 나뉘어진 두개의 element를 위해 insert
columns[0].insert(4,'FullName')
csv_data_3 = csv_data_2.map(lambda line: [(columns[0][i], linedata) for i, linedata in enumerate(line)])

csv_data_3.take(50)

[[('PassengerId', '1'),
  ('Survived', '0'),
  ('Pclass', '3'),
  ('Name', '"Braund'),
  ('FullName', ' Mr. Owen Harris"'),
  ('Sex', 'male'),
  ('Age', '22'),
  ('SibSp', '1'),
  ('Parch', '0'),
  ('Ticket', 'A/5 21171'),
  ('Fare', '7.25'),
  ('Cabin', ''),
  ('Embarked', 'S')],
 [('PassengerId', '2'),
  ('Survived', '1'),
  ('Pclass', '1'),
  ('Name', '"Cumings'),
  ('FullName', ' Mrs. John Bradley (Florence Briggs Thayer)"'),
  ('Sex', 'female'),
  ('Age', '38'),
  ('SibSp', '1'),
  ('Parch', '0'),
  ('Ticket', 'PC 17599'),
  ('Fare', '71.2833'),
  ('Cabin', 'C85'),
  ('Embarked', 'C')],
 [('PassengerId', '3'),
  ('Survived', '1'),
  ('Pclass', '3'),
  ('Name', '"Heikkinen'),
  ('FullName', ' Miss. Laina"'),
  ('Sex', 'female'),
  ('Age', '26'),
  ('SibSp', '0'),
  ('Parch', '0'),
  ('Ticket', 'STON/O2. 3101282'),
  ('Fare', '7.925'),
  ('Cabin', ''),
  ('Embarked', 'S')],
 [('PassengerId', '4'),
  ('Survived', '1'),
  ('Pclass', '1'),
  ('Name', '"Futrelle'),
  ('FullName', ' Mrs.

In [32]:
# 생존자와 사망자의 연령 총합 구하기
csv_data_4 = csv_data_3.map(lambda line:(line[1][1], line[6][1]))   # (생존여부, 연령)
# csv_data_4.take(50)
csv_data_4 = csv_data_4.filter(lambda tup : tup[1]!='' ) # age 결측치 제거
csv_data_4.take(50)

[('0', '22'),
 ('1', '38'),
 ('1', '26'),
 ('1', '35'),
 ('0', '35'),
 ('0', '54'),
 ('0', '2'),
 ('1', '27'),
 ('1', '14'),
 ('1', '4'),
 ('1', '58'),
 ('0', '20'),
 ('0', '39'),
 ('0', '14'),
 ('1', '55'),
 ('0', '2'),
 ('0', '31'),
 ('0', '35'),
 ('1', '34'),
 ('1', '15'),
 ('1', '28'),
 ('0', '8'),
 ('1', '38'),
 ('0', '19'),
 ('0', '40'),
 ('0', '66'),
 ('0', '28'),
 ('0', '42'),
 ('0', '21'),
 ('0', '18'),
 ('1', '14'),
 ('0', '40'),
 ('0', '27'),
 ('1', '3'),
 ('1', '19'),
 ('0', '18'),
 ('0', '7'),
 ('0', '21'),
 ('1', '49'),
 ('1', '29'),
 ('0', '65'),
 ('1', '21'),
 ('0', '28.5'),
 ('1', '5'),
 ('0', '11'),
 ('0', '22'),
 ('1', '38'),
 ('0', '45'),
 ('0', '4'),
 ('1', '29')]

In [35]:
#생존자, 사망자 나이 총합
age_sum_data = csv_data_4.reduceByKey(lambda accum, age: float(accum) + float(age))  
age_sum = age_sum_data.collect()
age_sum

[('0', 12985.5), ('1', 8219.67)]

In [36]:
# 생존자와 사망자의 사람 수 구하기
csv_data_5 = csv_data_3.map(lambda line:(line[1][1], 1))
survived_data = csv_data_5.reduceByKey(lambda accum, count: int(accum) + int(count)) 
survived_count = survived_data.collect()

In [37]:
age_sum_dict = dict(age_sum)
survived_dict = dict(survived_count)
age_sum_dict['1']

8219.67

In [38]:
age_sum_dict['0']

12985.5

In [40]:
avg_age_survived = age_sum_dict['1']/survived_dict['1']
print('생존자 평균 연령:' ,avg_age_survived)


생존자 평균 연령: 24.034122807017543


In [41]:
avg_age_died = age_sum_dict['0']/survived_dict['0']
print('사망자 평균 연령:' ,avg_age_died)

사망자 평균 연령: 23.653005464480874
